In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib

In [2]:
import geopandas as gpd
import geopandas

In [3]:
 'geopandas' in sys.modules # check geopandas module

True

In [4]:
sys.executable # check running enviornment

'C:\\Users\\34639\\anaconda3\\envs\\geo_env\\python.exe'

In [5]:
from platform import python_version

In [6]:
print(python_version()) # python version


3.8.10


In [7]:
# reading the geopackage
cadastral_ref_gdf = gpd.read_file("Ciclica_test_gis.gpkg", layer='CadastralReferences')


C:\Users\34639\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [8]:
cadastral_ref_gdf.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry
0,63939911378595,Residencial,505.0,505.0,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563..."
1,93428957957876,AltresNoCal,265.0,0.0,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147..."
2,91748101647838,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171..."
3,38225472390951,Emmagatzematge,22.0,0.0,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583..."
4,66150954127328,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966..."


In [9]:
# now we have a Geodataframe with valid geometry
type(cadastral_ref_gdf)

geopandas.geodataframe.GeoDataFrame

In [10]:
cadastral_ref_gdf.shape

(35991, 6)

In [11]:
cadastral_ref_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35991 entries, 0 to 35990
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   parcela_catastral  35991 non-null  object  
 1   immb_us_prn        35991 non-null  object  
 2   sup_total          35991 non-null  float64 
 3   sup_viv_sbr        35991 non-null  float64 
 4   demanda_cal_cte    23600 non-null  float64 
 5   geometry           35991 non-null  geometry
dtypes: float64(3), geometry(1), object(2)
memory usage: 1.6+ MB


In [12]:
cadastral_ref_gdf.describe() 

,sup_total,sup_viv_sbr,demanda_cal_cte
count,35991.000000,35991.000000,23600.000000
mean,425.627907,179.248159,6828.191080
std,1470.757165,373.147257,3876.109340
min,2.000000,0.000000,100.319108
25%,124.000000,54.000000,3460.362589
50%,213.000000,123.000000,6853.216876
75%,351.000000,198.000000,10179.524609
max,116159.000000,13389.000000,13499.576744


One time observed the main characteristics we will proceed with the exercices.

## 1. Assign census code id (‘id’ )  to every cadastral reference ( ‘parcela_catastral’).

As we don't have the id field we will have to obtain it form the data table CensusAreas.

In [13]:
census_ares_gdf = gpd.read_file('Ciclica_test_gis.gpkg', layer='CensusAreas')

C:\Users\34639\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [14]:
census_ares_gdf.head()

,id,geometry
0,0700201001,"MULTIPOLYGON (((4.19981 39.97058, 4.20001 39.9..."
1,0700202001,"MULTIPOLYGON (((4.13977 39.93362, 4.13977 39.9..."
2,0700202003,"MULTIPOLYGON (((4.13677 39.86414, 4.13657 39.8..."
3,0701501001,"MULTIPOLYGON (((3.84119 40.00544, 3.84063 40.0..."
4,0701501003,"MULTIPOLYGON (((3.83554 40.00015, 3.83341 39.9..."


In [15]:
census_ares_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        52 non-null     object  
 1   geometry  52 non-null     geometry
dtypes: geometry(1), object(1)
memory usage: 960.0+ bytes


#### What to keep in mind?

**Coordinate Reference System (CRS): the projection of the data.**

Every cadastral_ref_gdf Multipolygon is contained in a census_area_gdf Multipolygon, and how there isn't a common field we have to do a **geospatial join** and for **one** census area witt be **many** cadastral_ref_gdf["parcela_catastral"]. The final table has to have the same rows as cadastral_ref_gdf-the null geometris(0).

[Spatial Joins Geopandas](https://geopandas.org/en/stable/gallery/spatial_joins.html)

https://geopandas.org/en/stable/docs/user_guide/mergingdata.html

[My projects](https://github.com/JosepAnSabate/jupyter_maps/tree/main/earth_data_science/2_Vector_data_py/processing_vector_data)

In [16]:
cadastral_ref_gdf.crs == census_ares_gdf.crs

False

In [17]:
# View CRS of cadastral_ref_gdf
#  Projected coordinate Systems: coordinate systems that are localized to minimize visual distortion 
# in a particular region (e.g. Robinson, UTM, State Plane)
cadastral_ref_gdf.crs

<Projected CRS: EPSG:25831>
Name: ETRS89 / UTM zone 31N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 0°E and 6°E: Andorra; Belgium - onshore and offshore; Denmark - offshore; Germany - offshore; Jan Mayen - offshore; Norway including Svalbard - onshore and offshore; Spain - onshore and offshore.
- bounds: (0.0, 37.0, 6.01, 82.41)
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [18]:
# View CRS of census_ares_gdf, Geographic coordinate systems: coordinate systems
# that span the entire globe (e.g. latitude / longitude).
census_ares_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

We need to reproject one geodataframe. in this case given that we are working about Barcelona and the region is not much bigger 
we will reproject **census_ares_gdf to ETRS89 / UTM zone 31N.**

In [19]:
# Reproject the census_ares_gdf
census_ares_gdf_rep  = census_ares_gdf.to_crs(epsg=25831)

In [20]:
cadastral_ref_gdf.crs == census_ares_gdf_rep.crs

True

In a LEFT OUTER JOIN (how='left'), we keep all rows from the left and duplicate them if necessary to represent multiple hits between the two dataframes. Spatial Joins Geopandas. 

In [21]:
from shapely.geometry import Polygon, LineString, Point 

In [22]:
type(cadastral_ref_gdf)

geopandas.geodataframe.GeoDataFrame

In [23]:
type(census_ares_gdf)

geopandas.geodataframe.GeoDataFrame

[Source](https://github.com/JosepAnSabate/jupyter_maps/blob/main/earth_data_science/2_Vector_data_py/processing_vector_data/8_How%20to%20Join%20Attributes%20From%20One%20Shapefile%20to%20Another%20in%20Open%20Source%20Python%20Using%20Geopandas.ipynb)

In [24]:
# spatial join  between two GeoDataFrames
join_cadastral_gdf = gpd.sjoin(cadastral_ref_gdf, census_ares_gdf_rep,how="left", op="within")
join_cadastral_gdf.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id
0,63939911378595,Residencial,505.0,505.0,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563...",1.0,0700202001
1,93428957957876,AltresNoCal,265.0,0.0,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147...",37.0,0700201002
2,91748101647838,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171...",37.0,0700201002
3,38225472390951,Emmagatzematge,22.0,0.0,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583...",1.0,0700202001
4,66150954127328,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966...",37.0,0700201002


In [25]:
cadastral_ref_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35991 entries, 0 to 35990
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   parcela_catastral  35991 non-null  object  
 1   immb_us_prn        35991 non-null  object  
 2   sup_total          35991 non-null  float64 
 3   sup_viv_sbr        35991 non-null  float64 
 4   demanda_cal_cte    23600 non-null  float64 
 5   geometry           35991 non-null  geometry
dtypes: float64(3), geometry(1), object(2)
memory usage: 1.6+ MB


In [26]:
join_cadastral_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 35991 entries, 0 to 35990
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   parcela_catastral  35991 non-null  object  
 1   immb_us_prn        35991 non-null  object  
 2   sup_total          35991 non-null  float64 
 3   sup_viv_sbr        35991 non-null  float64 
 4   demanda_cal_cte    23600 non-null  float64 
 5   geometry           35991 non-null  geometry
 6   index_right        35139 non-null  float64 
 7   id                 35139 non-null  object  
dtypes: float64(4), geometry(1), object(3)
memory usage: 2.5+ MB


**Now we have the join gdf with the same rows and we don't have any non null values from id or geometry.**

In [27]:
join_cadastral_gdf.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id
0,63939911378595,Residencial,505.0,505.0,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563...",1.0,0700202001
1,93428957957876,AltresNoCal,265.0,0.0,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147...",37.0,0700201002
2,91748101647838,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171...",37.0,0700201002
3,38225472390951,Emmagatzematge,22.0,0.0,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583...",1.0,0700202001
4,66150954127328,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966...",37.0,0700201002


In [28]:
# writing a shp
join_cadastral_gdf.to_file("shp_res/cadastral_references_with_census_id.shp")

C:\Users\34639\AppData\Local\Temp/ipykernel_17272/569404371.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  join_cadastral_gdf.to_file("shp_res/cadastral_references_with_census_id.shp")


In [29]:
# plotting maps
import matplotlib.pyplot as plt
from shapely.geometry import box

## 2. The weighted average of energy demand (‘demanda_cal_cte’), by residential surface ( ‘sup_viv_sbr’), for all census areas(id).

[Numpy average](https://numpy.org/doc/stable/reference/generated/numpy.average.html)

In [30]:
join_cadastral_gdf.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id
0,63939911378595,Residencial,505.0,505.0,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563...",1.0,0700202001
1,93428957957876,AltresNoCal,265.0,0.0,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147...",37.0,0700201002
2,91748101647838,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171...",37.0,0700201002
3,38225472390951,Emmagatzematge,22.0,0.0,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583...",1.0,0700202001
4,66150954127328,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966...",37.0,0700201002


In [31]:
print('Null values from demanda_cal_cte:', join_cadastral_gdf[join_cadastral_gdf["demanda_cal_cte"].isna()].shape[0])

Null values from demanda_cal_cte: 12391


We will delete this rows from the gdf:

In [32]:
join_cadastral_gdf_not_na_demanda = join_cadastral_gdf[join_cadastral_gdf['demanda_cal_cte'].notna()]
join_cadastral_gdf.shape[0]

35991

In [33]:
print('Null values from id:', join_cadastral_gdf[join_cadastral_gdf["id"].isna()].shape[0])

Null values from id: 852


In [34]:
join_cadastral_gdf_not_na = join_cadastral_gdf_not_na_demanda[join_cadastral_gdf_not_na_demanda['id'].notna()]

In [35]:
print('Null values from id:', join_cadastral_gdf_not_na[join_cadastral_gdf_not_na["id"].isna()].shape[0], 'Rows gdf: ', join_cadastral_gdf_not_na.shape[0])

Null values from id: 0 Rows gdf:  23318


#### First we will calculate the average and next the weighted average.

In [36]:
# grouping by census id
join_cadastral_by_id = join_cadastral_gdf_not_na.groupby(["id"]).sum()
join_cadastral_by_id = join_cadastral_by_id.drop(['index_right'], axis=1) # axis 1 = col, drop col
join_cadastral_by_id.head()

,sup_total,sup_viv_sbr,demanda_cal_cte
id,,,
0700201001,150161.0,88070.33651,2.952605e+06
0700201002,296971.0,187863.00000,5.137823e+06
0700202001,134686.0,79854.00000,2.393955e+06
0700202002,167108.0,95691.00000,3.892483e+06
0700202003,233599.0,143203.00000,7.024901e+06


In [37]:
# first we will calculate the caloric demand by surface as a new col
join_cadastral_by_id["demand_cal_by_surf"] = join_cadastral_by_id["demanda_cal_cte"] / join_cadastral_by_id["sup_viv_sbr"]

In [38]:
join_cadastral_by_id.head() # now we have simple average by census

,sup_total,sup_viv_sbr,demanda_cal_cte,demand_cal_by_surf
id,,,,
0700201001,150161.0,88070.33651,2.952605e+06,33.525538
0700201002,296971.0,187863.00000,5.137823e+06,27.348775
0700202001,134686.0,79854.00000,2.393955e+06,29.979148
0700202002,167108.0,95691.00000,3.892483e+06,40.677626
0700202003,233599.0,143203.00000,7.024901e+06,49.055544


In [39]:
join_cadastral_gdf_not_na.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id
630,09354816997406,Emmagatzematge,153.0,66.0,10625.577393,"MULTIPOLYGON (((598647.353 4420785.924, 598658...",13.0,0700202002
631,97280445168223,Residencial,96.0,67.0,13210.883503,"MULTIPOLYGON (((598658.037 4420761.236, 598658...",13.0,0700202002
647,04951797057711,Residencial,35.0,20.0,860.320565,"MULTIPOLYGON (((598733.344 4420526.184, 598731...",13.0,0700202002
649,72509208715984,Residencial,42.0,42.0,4364.657211,"MULTIPOLYGON (((598818.556 4420501.117, 598816...",13.0,0700202002
1101,67546632942112,Residencial,47.0,34.0,11937.792683,"MULTIPOLYGON (((598627.640 4420723.104, 598681...",13.0,0700202002


In [40]:
join_cadastral_gdf_not_na.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 23318 entries, 630 to 35989
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   parcela_catastral  23318 non-null  object  
 1   immb_us_prn        23318 non-null  object  
 2   sup_total          23318 non-null  float64 
 3   sup_viv_sbr        23318 non-null  float64 
 4   demanda_cal_cte    23318 non-null  float64 
 5   geometry           23318 non-null  geometry
 6   index_right        23318 non-null  float64 
 7   id                 23318 non-null  object  
dtypes: float64(4), geometry(1), object(3)
memory usage: 1.6+ MB


In [41]:
join_cadastral_gdf_not_na["demand_cal_by_surf_wa"] = join_cadastral_gdf_not_na["demanda_cal_cte"] / join_cadastral_gdf_not_na["sup_viv_sbr"] 
# creating the new col weighted avg

C:\Users\34639\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [42]:
join_cadastral_gdf_not_na.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 23318 entries, 630 to 35989
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   parcela_catastral      23318 non-null  object  
 1   immb_us_prn            23318 non-null  object  
 2   sup_total              23318 non-null  float64 
 3   sup_viv_sbr            23318 non-null  float64 
 4   demanda_cal_cte        23318 non-null  float64 
 5   geometry               23318 non-null  geometry
 6   index_right            23318 non-null  float64 
 7   id                     23318 non-null  object  
 8   demand_cal_by_surf_wa  23318 non-null  float64 
dtypes: float64(5), geometry(1), object(3)
memory usage: 1.8+ MB


Why at the end of this exercise first I have some inf values in weighted average column sum when I obtain the result of this execise. **We will going to check some things.**

In [43]:
prova = join_cadastral_gdf_not_na[join_cadastral_gdf_not_na.id == "0703701003"] # query bi id, this id is a nan value in weighted average sum
prova

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id,demand_cal_by_surf_wa
18947,78041812125715,Residencial,391.0,189.0,4277.090375,"MULTIPOLYGON (((600456.436 4431035.440, 600454...",47.0,0703701003,22.630108
18948,42808757259790,Residencial,1198.0,1127.0,268.198504,"MULTIPOLYGON (((600402.694 4431035.555, 600404...",47.0,0703701003,0.237976
18949,82804963501534,Residencial,136.0,83.0,2012.825655,"MULTIPOLYGON (((600430.569 4430967.906, 600429...",47.0,0703701003,24.250912
18950,35092416558852,Residencial,3093.0,2664.0,273.995865,"MULTIPOLYGON (((600438.967 4430794.494, 600438...",47.0,0703701003,0.102851
18951,42230771658228,Comercial,839.0,267.0,8928.156931,"MULTIPOLYGON (((600456.075 4430759.615, 600466...",47.0,0703701003,33.438790
...,...,...,...,...,...,...,...,...,...
21398,46848461079173,Residencial,199.0,109.0,3905.480579,"MULTIPOLYGON (((599827.993 4430248.005, 599863...",47.0,0703701003,35.830097
21399,21821670879674,Residencial,242.0,137.0,11550.880412,"MULTIPOLYGON (((599827.963 4430250.305, 599827...",47.0,0703701003,84.312996
21400,39668805432277,Residencial,227.0,123.0,11143.220756,"MULTIPOLYGON (((599827.802 4430288.084, 599862...",47.0,0703701003,90.595291
21401,80312410274464,Residencial,256.0,180.0,1986.837940,"MULTIPOLYGON (((599827.676 4430327.575, 599827...",47.0,0703701003,11.037989


In [44]:
prova_df = pd.DataFrame(prova)

In [45]:
prova_df.describe() # inf values!!! 

,sup_total,sup_viv_sbr,demanda_cal_cte,index_right,demand_cal_by_surf_wa
count,1284.000000,1284.000000,1284.000000,1284.0,1284.000000
mean,460.327103,317.282355,6823.805350,47.0,inf
std,971.056343,663.929448,3852.050082,0.0,NaN
min,21.000000,0.000000,105.183408,47.0,0.056492
25%,169.000000,109.000000,3463.724613,47.0,15.916506
50%,230.500000,151.000000,6804.075930,47.0,38.519836
75%,348.000000,221.000000,9981.701281,47.0,67.748704
max,16797.000000,6810.000000,13499.576744,47.0,inf


In [46]:
# select the finit values
prova_df_r = prova_df[prova_df['demand_cal_by_surf_wa'] < 100000000]

In [47]:
prova_df_r.describe() 

,sup_total,sup_viv_sbr,demanda_cal_cte,index_right,demand_cal_by_surf_wa
count,1281.000000,1281.000000,1281.000000,1281.0,1281.000000
mean,460.811085,318.025405,6811.051613,47.0,48.592172
std,972.139201,664.529119,3847.086655,0.0,44.440665
min,21.000000,21.000000,105.183408,47.0,0.056492
25%,169.000000,110.000000,3461.989519,47.0,15.884080
50%,231.000000,151.000000,6793.766412,47.0,38.447331
75%,348.000000,221.000000,9944.839556,47.0,67.687602
max,16797.000000,6810.000000,13499.576744,47.0,465.420345


In [48]:
prova_df_r.shape[0] # We have lost the 3 inf values

1281

### Applyng a filter for selecting finite values in the gdf


In [49]:
join_cadastral_gdf_not_na.shape[0]

23318

In [50]:
join_cadastral_gdf_not_na_not_inf= join_cadastral_gdf_not_na[join_cadastral_gdf_not_na['demand_cal_by_surf_wa'] < 99999999] # selecting the finite values
join_cadastral_gdf_not_na_not_inf.describe() # Now we don't have the infinite values.

,sup_total,sup_viv_sbr,demanda_cal_cte,index_right,demand_cal_by_surf_wa
count,23304.000000,23304.000000,23304.000000,23304.000000,23304.000000
mean,370.477987,237.320180,6825.351596,26.881780,53.849174
std,743.930448,434.533576,3876.656849,15.202835,80.400242
min,10.000000,1.000000,100.319108,0.000000,0.056492
25%,154.000000,105.000000,3458.095097,13.000000,17.460398
50%,228.000000,152.000000,6845.450482,31.000000,39.234411
75%,345.250000,225.000000,10177.991737,40.000000,71.405364
max,28729.000000,13389.000000,13499.576744,51.000000,4256.376605


In [51]:
join_cadastral_gdf.shape[0] 

35991

We have droped the rows with inf values in the demand_cal_by_surf_wa field.

In [52]:
join_cadastral_wa = join_cadastral_gdf_not_na_not_inf.groupby(["id"]).mean() # grop by id and mean of weighted average.

In [53]:
join_cadastral_wa.head()

,sup_total,sup_viv_sbr,demanda_cal_cte,index_right,demand_cal_by_surf_wa
id,,,,,
0700201001,342.052392,200.615801,6725.752612,0.0,52.053273
0700201002,395.434088,250.150466,6841.308785,37.0,49.964210
0700202001,384.817143,228.154286,6839.871114,1.0,42.953922
0700202002,308.317343,176.551661,7181.702395,13.0,59.802629
0700202003,226.795146,139.032039,6820.292332,2.0,79.414523


In [54]:
join_cadastral_wa["average_no_weighted"] =  join_cadastral_by_id["demand_cal_by_surf"] # add average no weighted from previous df

In [55]:
join_cadastral_wa.head()

,sup_total,sup_viv_sbr,demanda_cal_cte,index_right,demand_cal_by_surf_wa,average_no_weighted
id,,,,,,
0700201001,342.052392,200.615801,6725.752612,0.0,52.053273,33.525538
0700201002,395.434088,250.150466,6841.308785,37.0,49.964210,27.348775
0700202001,384.817143,228.154286,6839.871114,1.0,42.953922,29.979148
0700202002,308.317343,176.551661,7181.702395,13.0,59.802629,40.677626
0700202003,226.795146,139.032039,6820.292332,2.0,79.414523,49.055544


Now we have the **weighted average** and the **average not weighted** grouped by census id (id).

### Final result

In [56]:
# 3 decimals positions
pd.options.display.float_format = '{:.2f}'.format

In [57]:
average_non_weighted = join_cadastral_wa.iloc[:, -2] # selecting rows from the df
average_non_weighted

id
0700201001   52.05
0700201002   49.96
0700202001   42.95
0700202002   59.80
0700202003   79.41
0701501001   57.34
0701501002   41.05
0701501003   33.55
0701501004   56.02
0701501005   58.38
0701502001   54.85
0701502002   43.09
0701502003   39.13
0701502004   36.89
0701503001   51.64
0701503002   42.73
0701503003   34.99
0701504001   45.43
0701504003   52.53
0701504004   62.23
0702301001   44.88
0702301002   26.70
0702301003   35.58
0703201001   67.20
0703201002   50.32
0703202001   43.63
0703202002   41.91
0703202003   44.74
0703202004   33.39
0703203001   42.73
0703203002   42.59
0703203003   68.66
0703203004    9.27
0703203005   16.48
0703203006   45.35
0703204001   48.47
0703204002   37.78
0703205001   68.00
0703701001   46.75
0703701002   50.26
0703701003   48.59
0705201001   45.23
0705201002   44.06
0705202001   88.38
0705202002   69.91
0706401001   45.90
0706401002   64.63
0706401003   59.25
0706401004   55.90
0706401005   38.14
0790201001   46.22
Name: demand_cal_by_surf_wa,

In [58]:
average_non_weighted.to_excel("./taules_exerc_gis/weighted_avg_of_energdeamnd_bysurface_ex_2.xlsx", sheet_name='Fulla1') 

## 3. Total surface by census area and property use category.

[Source](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.area.html)

Now we are working with ETRS so we can calculate in meters.

In [59]:
join_cadastral_gdf.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id
0,63939911378595,Residencial,505.00,505.00,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563...",1.00,0700202001
1,93428957957876,AltresNoCal,265.00,0.00,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147...",37.00,0700201002
2,91748101647838,Emmagatzematge,9.00,0.00,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171...",37.00,0700201002
3,38225472390951,Emmagatzematge,22.00,0.00,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583...",1.00,0700202001
4,66150954127328,Emmagatzematge,9.00,0.00,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966...",37.00,0700201002


In [60]:
join_cadastral_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 35991 entries, 0 to 35990
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   parcela_catastral  35991 non-null  object  
 1   immb_us_prn        35991 non-null  object  
 2   sup_total          35991 non-null  float64 
 3   sup_viv_sbr        35991 non-null  float64 
 4   demanda_cal_cte    23600 non-null  float64 
 5   geometry           35991 non-null  geometry
 6   index_right        35139 non-null  float64 
 7   id                 35139 non-null  object  
dtypes: float64(4), geometry(1), object(3)
memory usage: 2.5+ MB


In [61]:
join_cadastral_gdf.shape[0]

35991

For this exercice we will delete the na id and geometry

In [62]:
join_cadastral_gdf_not_na_id = join_cadastral_gdf[join_cadastral_gdf['id'].notna()] # droping not na values by column

In [63]:
join_cadastral_gdf_not_na_id_geom =  join_cadastral_gdf_not_na_id[join_cadastral_gdf_not_na_id['geometry'].notna()]

In [64]:
join_cadastral_gdf_not_na_id_geom.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 35139 entries, 0 to 35990
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   parcela_catastral  35139 non-null  object  
 1   immb_us_prn        35139 non-null  object  
 2   sup_total          35139 non-null  float64 
 3   sup_viv_sbr        35139 non-null  float64 
 4   demanda_cal_cte    23318 non-null  float64 
 5   geometry           35139 non-null  geometry
 6   index_right        35139 non-null  float64 
 7   id                 35139 non-null  object  
dtypes: float64(4), geometry(1), object(3)
memory usage: 2.4+ MB


In [65]:
join_cadastral_gdf_not_na_id_geom.shape[0]

35139

In [66]:
join_cadastral_gdf_not_na_id_geom.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id
0,63939911378595,Residencial,505.00,505.00,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563...",1.00,0700202001
1,93428957957876,AltresNoCal,265.00,0.00,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147...",37.00,0700201002
2,91748101647838,Emmagatzematge,9.00,0.00,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171...",37.00,0700201002
3,38225472390951,Emmagatzematge,22.00,0.00,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583...",1.00,0700202001
4,66150954127328,Emmagatzematge,9.00,0.00,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966...",37.00,0700201002


First we will calculate the surface of every parcela_catastral and next we will group by the requirements and the surfaces.

In [67]:
join_cadastral_gdf_not_na_id_geom.crs #   Projected coordinate Systems, we can calculate areas

<Projected CRS: EPSG:25831>
Name: ETRS89 / UTM zone 31N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 0°E and 6°E: Andorra; Belgium - onshore and offshore; Denmark - offshore; Germany - offshore; Jan Mayen - offshore; Norway including Svalbard - onshore and offshore; Spain - onshore and offshore.
- bounds: (0.0, 37.0, 6.01, 82.41)
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [68]:
join_cadastral_gdf_not_na_id_geom["area"] = join_cadastral_gdf_not_na_id_geom.area # calculating the area

In [69]:
join_cadastral_gdf_not_na_id_geom.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry,index_right,id,area
0,63939911378595,Residencial,505.00,505.00,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563...",1.00,0700202001,315.09
1,93428957957876,AltresNoCal,265.00,0.00,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147...",37.00,0700201002,269.06
2,91748101647838,Emmagatzematge,9.00,0.00,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171...",37.00,0700201002,9.04
3,38225472390951,Emmagatzematge,22.00,0.00,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583...",1.00,0700202001,22.69
4,66150954127328,Emmagatzematge,9.00,0.00,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966...",37.00,0700201002,8.86


In [70]:
join_cadastral_gdf_not_na_id_geom.crs

<Projected CRS: EPSG:25831>
Name: ETRS89 / UTM zone 31N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 0°E and 6°E: Andorra; Belgium - onshore and offshore; Denmark - offshore; Germany - offshore; Jan Mayen - offshore; Norway including Svalbard - onshore and offshore; Spain - onshore and offshore.
- bounds: (0.0, 37.0, 6.01, 82.41)
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

The distances are in **square meters** because we are working with a gdf with Projected coordinate System.

In [71]:
total_surface_by_2_cond = join_cadastral_gdf_not_na_id_geom.groupby(['id', 'immb_us_prn'])["area"].sum() #gfroupib by two condicions and sum area

### Results exercice 3

In [72]:
total_surface_by_2_cond.head(50)

id          immb_us_prn        
0700201001  AltresCal                 2174.37
            AltresNoCal            2226483.66
            Aparcament               60589.42
            Comercial                 1080.91
            Emmagatzematge          377304.37
            HotelerRestauracio       18983.22
            Industrial                3984.81
            IndustrialResta          10184.10
            Oficines                   160.60
            Piscines                   612.20
            Public                   22955.56
            Residencial            1494245.14
0700201002  AltresCal                  165.98
            AltresNoCal           12505368.92
            Aparcament               14292.61
            Comercial                28876.93
            Comu                       712.40
            Emmagatzematge          291945.72
            EnsenyamentCultural         63.44
            Esportiu                208215.70
            HotelerRestauracio      170790.07
  

In [73]:
total_surface_by_2_cond.to_excel("./taules_exerc_gis/surfaces_square_meters_by_id_census_and_use_category.xlsx", sheet_name='Fulla1') 

The main part of the buldings are classified as AltresNoCal.